Ideias:

New is always better (Heat map para ver se ando a ouvir musica que adicionei recentemente)

Divisão por estilos

Divisão por artistas

Horas ouvidas por dia

Horas mais frequentes a ouvir

Estilo mais predominante por hora

Caracteristicas da musica

In [8]:
# !pip install spotipy
# !pip install wordcloud
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install git+https://github.com/plamere/spotipy.git --upgrade
# !pip install seaborn
# !pip install python-dotenv
# !pip install openpyxl

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
from functions.spotipyTools import *
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns 
from dotenv import load_dotenv
import os
import openpyxl

In [10]:
load_dotenv()

True

In [11]:
e_client_id = os.environ["client_id"]
e_client_key = os.environ["client_key"]
e_playlist_id = os.environ["playlist_id"]
e_user_id = os.environ["user_id"]

sp_login = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=e_client_id,
                                               client_secret=e_client_key,
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-read-recently-played"))


client_credentials_manager = SpotifyClientCredentials(e_client_id, e_client_key)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
# sp_login.current_user_recently_played()

In [13]:
all_tracks = get_playlist_info(e_user_id, e_playlist_id, sp)

track_info_list = []

for item in all_tracks['track_id']:
  track_info_list.append(get_track_info(item, sp))

df_tracks = pd.DataFrame(list(zip(track_info_list,all_tracks['date_added'])) , columns=['name', 'artist_name', 'album_name', 'album_date', 'album_popularity', 'track_duration', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo','artist_url','date_added'])

ValueError: 13 columns passed, passed data had 2 columns

In [ ]:
#df_tracks['album_year'] = pd.to_datetime(df_tracks['album_year'], format='%Y-%m-%d').dt.year
df_tracks['album_year'] = pd.to_datetime(df_tracks['album_date'], format='%Y-%m-%d').dt.year

In [ ]:
track_minutes = np.floor(df_tracks['track_duration']/1000/60).astype(int).astype(str)
track_seconds = np.mod(df_tracks['track_duration']/1000,60).astype(int).astype(str)
track_time = track_minutes.append(track_seconds).groupby(level=0).agg(':'.join)
df_tracks['track_duration'] = pd.to_datetime(track_time, format='%M:%S').dt.time
df_tracks['track_duration_secs'] = df_tracks['track_duration'].apply(lambda x: x.minute * 60 + x.second)

In [ ]:
df_tracks.tail()

In [ ]:
artists = df_tracks['artist_url'].unique()
genres = get_artist_genre(artists, sp)

In [ ]:
exploded_genres = pd.Series(genres, name='Genre').explode().reset_index()
text = ' '.join(str(i) for i in exploded_genres.Genre)

In [ ]:
wordcloud = WordCloud(background_color='White').generate(text)

In [ ]:
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
df_tracks.to_excel('PlaylistTracks.xlsx', engine='openpyxl')